In [ ]:
import numpy as np
import re
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

In [ ]:
f = open("/content/book-war-and-peace.txt")
t = f.readlines()
t = ''.join(t)

In [ ]:
text = re.sub('[^a-zA-Z]', ' ', t)[:50000]
text = text.lower()

In [ ]:
d = {}
for i in range(0, 26):
    d[chr(i+97)] = i
d[' '] = 26

In [ ]:
V = np.zeros(50000)
for i in range(len(text)):
    V[i] = d[text[i]]

In [ ]:
V = V[0:100].astype(int)
print(V)

In [ ]:
a = np.array([[0.47468,0.52532],[0.51656,0.48344]])
print(a)

In [ ]:
b = np.array([[0.03735,0.03909 ],
[0.03408 ,0.03537 ],
[0.03455 ,0.03537 ],
[0.03828, 0.03909 ],
[0.03782, 0.03583 ],
[0.03922 ,0.03630 ],
[0.03688, 0.04048 ],
[0.03408, 0.03537 ],
[0.03875 ,0.03816 ],
[0.04062 ,0.03909 ],
[0.03735 ,0.03490 ],
[0.03968, 0.03723 ],
[0.03548 ,0.03537 ],
[0.03735 ,0.03909 ],
[0.04062, 0.03397 ],
[0.03595, 0.03397 ],
[0.03641, 0.03816 ],
[0.03408, 0.03676 ],
[0.04062 ,0.04048 ],
[0.03548, 0.03443 ],
[0.03922, 0.03537 ],
[0.04062 ,0.03955],
[0.03455, 0.03816 ],
[0.03595, 0.03723 ],
[0.03408,0.03769 ],
[0.03408 ,0.03955 ],
[0.03688, 0.03397 ]])
b = b.T
print(b.shape)

In [ ]:
pi = np.array([ 0.51316,0.48684])
print(pi[0],pi[1])

In [ ]:
print(V.shape)
print(a.shape)
print(b.shape)
print(V.shape[0], a.shape[0])

In [ ]:
N = 2
M = 27
MaxIters = 100
iters = 0
import math
oldLogProb = -math.inf

In [ ]:
T = len(V)
print(T)

In [ ]:
def War_peace(a,b,V,pi,N,M,T,oldLogProb):
# Alpha pass
    c = np.zeros([T, 1])
    alpha = np.zeros([T,N])

    for i in range(N):
        alpha[0][i] = pi[i]*b[i][V[0]]
        c[0][0] = c[0][0]+alpha[0][i]
    c[0][0] = 1/c[0][0]

    for i in range(N):
        alpha[0][i] = c[0][0]*alpha[0][i]

    for t in range(1,T):
        c[t][0] = 0
        for i in range(N):
            alpha[t][i]=0
            for j in range(N):
                alpha[t][i] = alpha[t][i]+ alpha[t-1][i]*a[j][i]
            alpha[t][i] = alpha[t][i]*b[i][V[t]]
            c[t][0] = c[t][0] + alpha[t][i]


        c[t][0] = 1/ c[t][0]


        for i in range(N):
            alpha[t][i] = c[t][0]*alpha[t][i]

    # print(c)
    beta = np.zeros([T,N])
    for i in range(N):
        beta[T-1][i] = c[T-1][0]

    for t in range(T-2,-1,-1):
        for i in range(N):
            beta[t][i] = 0
            for j in range(N):
                beta[t][i] = beta[t][i]+ a[i][j]*b[j][V[t+1]]*beta[t+1][j]
            beta[t][i] = c[t][0]*beta[t][i]



    digamma=  np.zeros([T,N,N])
    gamma =  np.zeros([T,N])
    for t in range(T-1):
        for i in range(N):
            gamma[t][i]=0
            for j in range(N):
                digamma[t][i][j] = alpha[t][i]*a[i][j]*b[j][V[t+1]]*beta[t+1][j]
                gamma[t][i] = gamma[t][i]+digamma[t][i][j]


    for i in range(N):
         gamma[T-1][i]= alpha[T-1][i]
    # print(gamma)
    # re-estimate π
    for i in range(N):
        pi[i] = gamma[0][i]
    #re-estimate A
    for i in range(N):
        denom = 0
        for t in range(T-1):
            denom = denom + gamma[t][i]
        for j in range(N):
            numer = 0
            for t in range(T-1):
                numer = numer + digamma[t][i][j]
            a[i][j] = numer/denom

    #re-estimate B
    for i in range(N):
        denom = 0
        for t in range(T):
            denom = denom + gamma[t][i]
        for j in range(M):
            numer = 0
            for t in range(T):
                if (V[t]==j):
                    numer = numer + gamma[t][i]
            b[i][j] = numer/denom
    import math
    logprob = 0
    for i in range(T):
        logprob = logprob + math.log(c[i][0])
    logprob = -1*logprob

    if logprob>oldLogProb:
        oldLogProb = logprob
#             War_peace(a,b,V,pi,N,M,T,oldLogProb)


    return a,b,pi,oldLogProb






In [ ]:
A,B,pie,P = War_peace(a,b,V,pi,N,M,T,oldLogProb)
for i in range(5):
    A,B,pie,P =  War_peace(A,B,V,pie,N,M,T,P)



print('A',A)
print('B',B)
print('PI',pie)
print("log(p(0/lamda))",P)


In [ ]:
A,B,pie,P = War_peace(a,b,V,pi,N,M,T,oldLogProb)
for i in range(5):
    A,B,pie,P =  War_peace(A,B,V,pie,N,M,T,P)



print('A',A)
print('B',B)
print('PI',pie)
print("log(p(0/lamda))",P)
